In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from sklearn.linear_model import LogisticRegression

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
pa = pd.read_csv(DATASET_DIR / "previous_application.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
n_heads = [element for element in pa.columns if pa[element].dtype.name == "object"]

In [5]:
pa = pd.merge(pa, app_train[["SK_ID_CURR","TARGET"]] ,on="SK_ID_CURR")

In [6]:
df = pa[n_heads]
df.head()

,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,middle,POS mobile with interest
1,Consumer loans,SATURDAY,Y,XAP,Refused,Cash through the bank,SCO,NaN,Repeater,Clothing and Accessories,POS,XNA,Regional / Local,Furniture,low_normal,POS industry with interest
2,Consumer loans,SUNDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,New,Furniture,POS,XNA,Stone,Furniture,middle,POS industry with interest
3,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,XNA,low_action,Cash X-Sell: low
4,Cash loans,WEDNESDAY,Y,XNA,Approved,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),XNA,low_normal,Cash X-Sell: low


In [7]:
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    rate = df[key].isnull().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
result

,header,rate,des


In [8]:
list(result.header)

[]

In [9]:
n_heads = [head for head in n_heads if head not in list(result.header)]
df.head()

,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,middle,POS mobile with interest
1,Consumer loans,SATURDAY,Y,XAP,Refused,Cash through the bank,SCO,NaN,Repeater,Clothing and Accessories,POS,XNA,Regional / Local,Furniture,low_normal,POS industry with interest
2,Consumer loans,SUNDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,New,Furniture,POS,XNA,Stone,Furniture,middle,POS industry with interest
3,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,XNA,low_action,Cash X-Sell: low
4,Cash loans,WEDNESDAY,Y,XNA,Approved,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),XNA,low_normal,Cash X-Sell: low


In [10]:
c = df[n_heads].corr(method='spearman') * 100

In [11]:
families = []
for i, row in c.iterrows():
    r = row[row > 70]
    if len(r) > 1 and set(r.index) not in families:
        print(r)
        print("\n")
        
        families.append(set(r.index))

In [12]:
result = {
          "family":[],
          "head":[],
          "r2":[],
          "na":[],
          "rate":[]
         }

for i, family in enumerate(families):
    headers = list(family)
    
    result["family"].append("")
    result["head"].append("")
    result["r2"].append("")
    result["na"].append("")
    result["rate"].append("")
    
    for head in headers:
        d = df[["TARGET"] + [head]]
        na = d[head].isna().sum() / len(d) * 100
        d = d.dropna()
        x = d[[head]]
        y = d[["TARGET"]]
        model = LogisticRegression().fit(x, y.values.ravel())
        r2 = round(model.score(x,y),5)
        
        result["family"].append(i)
        result["head"].append(head)
        result["r2"].append(round(r2,5))
        result["na"].append(round(na,2))
        result["rate"].append(r2/na)
    
        

In [13]:
result = pd.DataFrame(result)
result

,family,head,r2,na,rate


In [14]:
result = {
    "head":[],
    "des":[]
}

for head in n_heads:
    result["head"].append(head)
    result["des"].append(des[des["Row"] == head]["Description"])
    
result = pd.DataFrame(result)
result

,head,des
0,NAME_CONTRACT_TYPE,"5 Identification if loan is cash or revolving 178 Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application Name: Description, dtype: object"
1,WEEKDAY_APPR_PROCESS_START,"35 On which day of the week did the client apply for the loan 184 On which day of the week did the client apply for previous application Name: Description, dtype: object"
2,FLAG_LAST_APPL_PER_CONTRACT,"186 Flag if it was last application for the previous contract. Sometimes by mistake of client or our clerk there could be more applications for one single contract Name: Description, dtype: object"
3,NAME_CASH_LOAN_PURPOSE,"192 Purpose of the cash loan Name: Description, dtype: object"
4,NAME_CONTRACT_STATUS,"150 Contract status during the month 173 Contract status (active signed,...) on the previous credit 193 Contract status (approved, cancelled, ...) of previous application Name: Description, dtype: object"
5,NAME_PAYMENT_TYPE,"195 Payment method that client chose to pay for the previous application Name: Description, dtype: object"
6,CODE_REJECT_REASON,"196 Why was the previous application rejected Name: Description, dtype: object"
7,NAME_TYPE_SUITE,"14 Who was accompanying client when he was applying for the loan 197 Who accompanied client when applying for the previous application Name: Description, dtype: object"
8,NAME_CLIENT_TYPE,"198 Was the client old or new client when applying for the previous application Name: Description, dtype: object"
9,NAME_GOODS_CATEGORY,"199 What kind of goods did the client apply for in the previous application Name: Description, dtype: object"


In [15]:
remove = ["WEEKDAY_APPR_PROCESS_START", "FLAG_LAST_APPL_PER_CONTRACT", "NAME_TYPE_SUITE","NAME_PRODUCT_TYPE"]
n_heads = [head for head in n_heads if head not in remove]

In [16]:
df[n_heads].head()

,NAME_CONTRACT_TYPE,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,Consumer loans,XAP,Approved,Cash through the bank,XAP,Repeater,Mobile,POS,Country-wide,Connectivity,middle,POS mobile with interest
1,Consumer loans,XAP,Refused,Cash through the bank,SCO,Repeater,Clothing and Accessories,POS,Regional / Local,Furniture,low_normal,POS industry with interest
2,Consumer loans,XAP,Approved,Cash through the bank,XAP,New,Furniture,POS,Stone,Furniture,middle,POS industry with interest
3,Cash loans,XNA,Approved,XNA,XAP,Repeater,XNA,Cash,Contact center,XNA,low_action,Cash X-Sell: low
4,Cash loans,XNA,Approved,XNA,XAP,Repeater,XNA,Cash,AP+ (Cash loan),XNA,low_normal,Cash X-Sell: low


In [17]:
df = pa[["SK_ID_PREV", "SK_ID_CURR", "TARGET"] + n_heads]
df.head()

,SK_ID_PREV,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,2030495,271877,0,Consumer loans,XAP,Approved,Cash through the bank,XAP,Repeater,Mobile,POS,Country-wide,Connectivity,middle,POS mobile with interest
1,1696966,271877,0,Consumer loans,XAP,Refused,Cash through the bank,SCO,Repeater,Clothing and Accessories,POS,Regional / Local,Furniture,low_normal,POS industry with interest
2,2154916,271877,0,Consumer loans,XAP,Approved,Cash through the bank,XAP,New,Furniture,POS,Stone,Furniture,middle,POS industry with interest
3,2802425,108129,0,Cash loans,XNA,Approved,XNA,XAP,Repeater,XNA,Cash,Contact center,XNA,low_action,Cash X-Sell: low
4,1536272,108129,0,Cash loans,XNA,Approved,XNA,XAP,Repeater,XNA,Cash,AP+ (Cash loan),XNA,low_normal,Cash X-Sell: low


In [18]:
n_heads

['NAME_CONTRACT_TYPE',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [19]:
for index, head in enumerate(n_heads):
    a = df[["SK_ID_CURR", head]].groupby(by=["SK_ID_CURR", head]).size().unstack(fill_value=0)
    a.columns = [head + " " + el for el in list(a.columns)]
    if index == 0:
        result = a
    else:
        result = pd.merge(result, a, left_index=True, right_index=True)
df = result

In [20]:
df = pd.merge(df, app_train[["SK_ID_CURR","TARGET"]] ,on="SK_ID_CURR")
df = df.set_index("SK_ID_CURR")

In [21]:
df.head()

,NAME_CONTRACT_TYPE Cash loans,NAME_CONTRACT_TYPE Consumer loans,NAME_CONTRACT_TYPE Revolving loans,NAME_CONTRACT_TYPE XNA,NAME_CASH_LOAN_PURPOSE Building a house or an annex,NAME_CASH_LOAN_PURPOSE Business development,NAME_CASH_LOAN_PURPOSE Buying a garage,NAME_CASH_LOAN_PURPOSE Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE Buying a home,NAME_CASH_LOAN_PURPOSE Buying a new car,NAME_CASH_LOAN_PURPOSE Buying a used car,NAME_CASH_LOAN_PURPOSE Car repairs,NAME_CASH_LOAN_PURPOSE Education,NAME_CASH_LOAN_PURPOSE Everyday expenses,NAME_CASH_LOAN_PURPOSE Furniture,NAME_CASH_LOAN_PURPOSE Gasification / water supply,NAME_CASH_LOAN_PURPOSE Hobby,NAME_CASH_LOAN_PURPOSE Journey,NAME_CASH_LOAN_PURPOSE Medicine,NAME_CASH_LOAN_PURPOSE Money for a third person,NAME_CASH_LOAN_PURPOSE Other,NAME_CASH_LOAN_PURPOSE Payments on other loans,NAME_CASH_LOAN_PURPOSE Purchase of electronic equipment,NAME_CASH_LOAN_PURPOSE Refusal to name the goal,NAME_CASH_LOAN_PURPOSE Repairs,NAME_CASH_LOAN_PURPOSE Urgent needs,NAME_CASH_LOAN_PURPOSE Wedding / gift / holiday,NAME_CASH_LOAN_PURPOSE XAP,NAME_CASH_LOAN_PURPOSE XNA,NAME_CONTRACT_STATUS Approved,NAME_CONTRACT_STATUS Canceled,NAME_CONTRACT_STATUS Refused,NAME_CONTRACT_STATUS Unused offer,NAME_PAYMENT_TYPE Cash through the bank,NAME_PAYMENT_TYPE Cashless from the account of the employer,NAME_PAYMENT_TYPE Non-cash from your account,NAME_PAYMENT_TYPE XNA,CODE_REJECT_REASON CLIENT,CODE_REJECT_REASON HC,CODE_REJECT_REASON LIMIT,CODE_REJECT_REASON SCO,CODE_REJECT_REASON SCOFR,CODE_REJECT_REASON SYSTEM,CODE_REJECT_REASON VERIF,CODE_REJECT_REASON XAP,CODE_REJECT_REASON XNA,NAME_CLIENT_TYPE New,NAME_CLIENT_TYPE Refreshed,NAME_CLIENT_TYPE Repeater,NAME_CLIENT_TYPE XNA,NAME_GOODS_CATEGORY Additional Service,NAME_GOODS_CATEGORY Animals,NAME_GOODS_CATEGORY Audio/Video,NAME_GOODS_CATEGORY Auto Accessories,NAME_GOODS_CATEGORY Clothing and Accessories,NAME_GOODS_CATEGORY Computers,NAME_GOODS_CATEGORY Construction Materials,NAME_GOODS_CATEGORY Consumer Electronics,NAME_GOODS_CATEGORY Direct Sales,NAME_GOODS_CATEGORY Education,NAME_GOODS_CATEGORY Fitness,NAME_GOODS_CATEGORY Furniture,NAME_GOODS_CATEGORY Gardening,NAME_GOODS_CATEGORY Homewares,NAME_GOODS_CATEGORY Insurance,NAME_GOODS_CATEGORY Jewelry,NAME_GOODS_CATEGORY Medical Supplies,NAME_GOODS_CATEGORY Medicine,NAME_GOODS_CATEGORY Mobile,NAME_GOODS_CATEGORY Office Appliances,NAME_GOODS_CATEGORY Other,NAME_GOODS_CATEGORY Photo / Cinema Equipment,NAME_GOODS_CATEGORY Sport and Leisure,NAME_GOODS_CATEGORY Tourism,NAME_GOODS_CATEGORY Vehicles,NAME_GOODS_CATEGORY Weapon,NAME_GOODS_CATEGORY XNA,NAME_PORTFOLIO Cards,NAME_PORTFOLIO Cars,NAME_PORTFOLIO Cash,NAME_PORTFOLIO POS,NAME_PORTFOLIO XNA,CHANNEL_TYPE AP+ (Cash loan),CHANNEL_TYPE Car dealer,CHANNEL_TYPE Channel of corporate sales,CHANNEL_TYPE Contact center,CHANNEL_TYPE Country-wide,CHANNEL_TYPE Credit and cash offices,CHANNEL_TYPE Regional / Local,CHANNEL_TYPE Stone,NAME_SELLER_INDUSTRY Auto technology,NAME_SELLER_INDUSTRY Clothing,NAME_SELLER_INDUSTRY Connectivity,NAME_SELLER_INDUSTRY Construction,NAME_SELLER_INDUSTRY Consumer electronics,NAME_SELLER_INDUSTRY Furniture,NAME_SELLER_INDUSTRY Industry,NAME_SELLER_INDUSTRY Jewelry,NAME_SELLER_INDUSTRY MLM partners,NAME_SELLER_INDUSTRY Tourism,NAME_SELLER_INDUSTRY XNA,NAME_YIELD_GROUP XNA,NAME_YIELD_GROUP high,NAME_YIELD_GROUP low_action,NAME_YIELD_GROUP low_normal,NAME_YIELD_GROUP middle,PRODUCT_COMBINATION Card Street,PRODUCT_COMBINATION Card X-Sell,PRODUCT_COMBINATION Cash,PRODUCT_COMBINATION Cash Street: high,PRODUCT_COMBINATION Cash Street: low,PRODUCT_COMBINATION Cash Street: middle,PRODUCT_COMBINATION Cash X-Sell: high,PRODUCT_COMBINATION Cash X-Sell: low,PRODUCT_COMBINATION Cash X-Sell: middle,PRODUCT_COMBINATION POS household with interest,PRODUCT_COMBINATION POS household without interest,PRODUCT_COMBINATION POS industry with interest,PRODUCT_COMBINATION POS industry without interest,PRODUCT_COMBINATION POS mobile with interest,PRODUCT_COMBINATION POS mobile with

In [22]:
n_heads = list(df.columns)
n_heads = [head for head in n_heads if head not in ["TARGET"]]

In [42]:
payback = df[df["TARGET"] == 0]
default = df[df["TARGET"] == 1]

In [43]:
rates = []
heads = []
for head in n_heads:

    pays = payback[head].value_counts()
    defs = default[head].value_counts()

    pays = pays/pays.sum()*100
    defs = defs/defs.sum()*100
    diff = abs(pays-defs)
    d1 = max(diff.sort_values(ascending=False))
    try:
        diff = abs(pays[[i for i in pays.index if i not in defs.index]])
        d2 = max(diff.sort_values(ascending=False))
    except ValueError:
        d2 = 0
    try:
        diff = abs(defs[[i for i in defs.index if i not in pays.index]])
        d3 = max(diff.sort_values(ascending=False))
    except ValueError:
        d3 = 0
    rates.append(max([d1,d2,d3]))
    heads.append(head)
    
    result = {
          "Cats":heads,
          "Rate":rates
         }
result = pd.DataFrame(result).sort_values("Rate", ascending=False)
result

,Cats,Rate
31,NAME_CONTRACT_STATUS Refused,9.754297
38,CODE_REJECT_REASON HC,7.215312
104,NAME_YIELD_GROUP low_normal,6.929703
2,NAME_CONTRACT_TYPE Revolving loans,6.371934
106,PRODUCT_COMBINATION Card Street,6.168984
46,NAME_CLIENT_TYPE New,6.153663
102,NAME_YIELD_GROUP high,6.146903
77,NAME_PORTFOLIO Cards,6.032320
113,PRODUCT_COMBINATION Cash X-Sell: low,5.514512
47,NAME_CLIENT_TYPE Refreshed,5.249064


In [44]:
result[result["Rate"] >= 5]

,Cats,Rate
31,NAME_CONTRACT_STATUS Refused,9.754297
38,CODE_REJECT_REASON HC,7.215312
104,NAME_YIELD_GROUP low_normal,6.929703
2,NAME_CONTRACT_TYPE Revolving loans,6.371934
106,PRODUCT_COMBINATION Card Street,6.168984
46,NAME_CLIENT_TYPE New,6.153663
102,NAME_YIELD_GROUP high,6.146903
77,NAME_PORTFOLIO Cards,6.032320
113,PRODUCT_COMBINATION Cash X-Sell: low,5.514512
47,NAME_CLIENT_TYPE Refreshed,5.249064


In [45]:
n_heads = result[result["Rate"] >= 5].Cats.values

In [46]:
# transformiert kategorische variablen in integer
for head in n_heads:
    df[head], cats = pd.factorize(df[head])

In [47]:
df = df[n_heads]

In [48]:
df.head()

,NAME_CONTRACT_STATUS Refused,CODE_REJECT_REASON HC,NAME_YIELD_GROUP low_normal,NAME_CONTRACT_TYPE Revolving loans,PRODUCT_COMBINATION Card Street,NAME_CLIENT_TYPE New,NAME_YIELD_GROUP high,NAME_PORTFOLIO Cards,PRODUCT_COMBINATION Cash X-Sell: low,NAME_CLIENT_TYPE Refreshed,PRODUCT_COMBINATION POS industry with interest
SK_ID_CURR,,,,,,,,,,,
100002,0,0,0,0,0,0,0,0,0,0,0
100003,0,0,0,0,0,1,0,0,1,1,1
100004,0,0,1,0,0,0,0,0,0,0,0
100006,1,0,2,1,1,0,1,1,2,0,1
100007,0,0,1,0,0,0,2,0,0,0,0


In [49]:
df.to_csv(DATASET_DIR / "Datenaufbereitung" / "pa_cats.csv")